# ODE Solve
### scipy中有专门用于求解ode的函数odeint，下面的介绍主要改编自官方文献：https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html#scipy.integrate.odeint

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [ ]:
def pend(a, t, b, c):
    theta, omega = a
    dadt = [omega, -b*omega - c*np.sin(theta)]
    return dadt
#定义需要求解的方程，先手动化为一阶形式，注意自变量的顺序是：1待求解函数 2待求解函数的一阶导 3-n常数参数
#需要求解的函数有多个时用列表表示。高阶导数可以通过这种方式降为一阶导
#return待求解函数的导数

In [ ]:
import sympy

In [ ]:
b = 0.25
c = 5.0

a0 = [np.pi - 0.1, 0.0]

t = np.linspace(0, 10, 101)

sol = odeint(pend, a0, t, args=(b, c))
#odeint是求解ode的函数，自变量中，pend是之前def好的待求解函数导数，y0是待求解函数的初始值，t是函数的自变量
#args标记了之前def的pend有哪些自变量是常数参量
#这里sol的维度是(101,2)，对应了2个求解的函数在101个时间点的取值

In [ ]:
plt.plot(t, sol[:, 0], 'b', label='theta(t)')
plt.plot(t, sol[:, 1], 'g', label='omega(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

# Task 4

### 了解了odeint函数的用法之后，我们来尝试求解双摆的运动方程。双摆的示意图：
![](https://www.myphysicslab.com/pendulum/dbl_pendulum.gif)
### 我们需要求解 theta1和theta2随时间的变化。方程可以参考 https://www.myphysicslab.com/pendulum/double-pendulum-en.html

In [ ]:
def d_pend(y,t,L1,L2,m1,m2,g):
    the1, o1, the2, o2 = y
    dydt = np.zeros_like(y)
    dydt[0] = o1
    dydt[1] = (-g*(2*m1+m2)*np.sin(the1)-m2*g*np.sin(the1-2*the2)-2*np.sin(the1-the2)*m2*
               (o2**2*L2+o1**2*L1*np.cos(the1-the2)))/(L1*(2*m1+m2-m2*np.cos(2*the1-2*the2)))
    dydt[2] = o2
    dydt[3] = 2*np.sin(the1-the2)*(o1**2*L1*(m1+m2)+g*(m1+m2)*np.cos(the1) +o2**2*L2*m2
                                *np.cos(the1-the2))/(L2*(2*m1+m2-m2*np.cos(2*the1-2*the2)))
    #自己完成运动方程
    return dydt

In [ ]:
L1 = 1
L2 = 1
m1 = 1
m2 = 1
g = 9.8
#参数可以任意设置

y0 = [1,0,0,0]#自己定义一个初始位置

t = np.linspace(0, 20, 201)

d_sol = odeint(d_pend, y0, t, args=(L1,L2,m1,m2,g))

In [ ]:
#画出theta-t图
plt.plot(t, d_sol[:, 0], 'b', label='theta1(t)')
plt.plot(t, d_sol[:, 2], 'g', label='theta2(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

### 下面用动画画出双摆的动态轨迹，可能遇到报错，参考解决方案： https://stackoverflow.com/questions/13316397/matplotlib-animation-no-moviewriters-available

In [ ]:
import matplotlib.animation as animation

x1 = L1*np.sin(d_sol[:, 0])
y1 = -L1*np.cos(d_sol[:, 0])

x2 = L2*np.sin(d_sol[:, 2]) + x1
y2 = -L2*np.cos(d_sol[:, 2]) + y1

def animate(i):
    thisx = [0, x1[i], x2[i]]
    thisy = [0, y1[i], y2[i]]

    line.set_data(thisx, thisy)
    time_text.set_text(time_template % (i*0.1))
    return line, time_text

fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
ax.set_aspect('equal')
ax.grid()
#定义一个空的图像

line, = ax.plot([], [], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
#定义线条的样式和时间标签

def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text
#初始状态

ani = animation.FuncAnimation(fig, animate, np.arange(1, 200),
                              interval=25, blit=True, init_func=init)

ani.save('double_pendulum.mp4', fps=10)

In [ ]:
import io
import base64
from IPython.display import HTML
video = io.open('double_pendulum.mp4', 'r+b').read()
#在这里读取刚才生成的视频，注意文件名与之前保存的视频相对应
encoded = base64.b64encode(video)
HTML(data='''<video width="500" height="360" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))


## 附加题：拉格朗日方程
### 以下是双摆的拉格朗日方程求解代码，采用了sympy求解微分方程。熟悉以下代码后，列出拉格朗日方程后利用代码求解移动摆并画出轨迹。
### 代码修改自：http://bigsec.net/b52/scipydoc/double_pendulum.html

### 移动摆示意图
![](http://d2vlcm61l7u1fs.cloudfront.net/media%2F798%2F798f3234-8edd-4ee0-a583-a3e9ee1dc88c%2FphpzJEV3u.png)

In [ ]:
from sympy import *
from sympy import Derivative as D

In [ ]:
var("l_x1 l_x2 l_y1 l_y2 l1 l2 m1 m2 th1 th2 dth1 dth2 ddth1 ddth2 t g tmp")

sublist = [
(D(th1(t), t, t), ddth1),
(D(th1(t), t), dth1),
(D(th2(t), t, t), ddth2),
(D(th2(t),t), dth2),
(th1(t), th1),
(th2(t), th2)    
]

l_x1 = l1*sin(th1(t))
l_y1 = -l1*cos(th1(t))
l_x2 = l1*sin(th1(t)) + l2*sin(th2(t))
l_y2 = -l1*cos(th1(t)) - l2*cos(th2(t))

vx1 = diff(l_x1, t)
vx2 = diff(l_x2, t)
vy1 = diff(l_y1, t)
vy2 = diff(l_y2, t)

# 拉格朗日量
L = m1/2*(vx1**2 + vy1**2) + m2/2*(vx2**2 + vy2**2) - m1*g*l_y1 - m2*g*l_y2

# 拉格朗日方程
def lagrange_equation(L, v):    
    a = L.subs(D(v(t), t), tmp).diff(tmp).subs(tmp, D(v(t), t))
    b = L.subs(D(v(t), t), tmp).subs(v(t), v).diff(v).subs(v, v(t)).subs(tmp, D(v(t), t))
    c = a.diff(t) - b
    c = c.subs(sublist)  
    c = trigsimp(simplify(c))
    c = collect(c, [th1,th2,dth1,dth2,ddth1,ddth2])
    return c

eq1 = lagrange_equation(L, th1)
eq2 = lagrange_equation(L, th2)

In [ ]:
from math import sin,cos
import numpy as np
from scipy.integrate import odeint

g = 9.8

class DoublePendulum(object):
    def __init__(self, m1, m2, l1, l2):
        self.m1, self.m2, self.l1, self.l2 = m1, m2, l1, l2
        self.init_status = np.array([0.0,0.0,0.0,0.0])
        
    def equations(self, w, t):
        """
        微分方程公式
        """
        m1, m2, l1, l2 = self.m1, self.m2, self.l1, self.l2
        th1, th2, v1, v2 = w
        dth1 = v1
        dth2 = v2
        
        #eq of th1
        a = l1*l1*(m1+m2)  # dv1 parameter
        b = l1*m2*l2*cos(th1-th2) # dv2 paramter
        c = l1*(m2*l2*sin(th1-th2)*dth2*dth2 + (m1+m2)*g*sin(th1))
        
        #eq of th2
        d = m2*l2*l1*cos(th1-th2) # dv1 parameter
        e = m2*l2*l2 # dv2 parameter
        f = m2*l2*(-l1*sin(th1-th2)*dth1*dth1 + g*sin(th2))
        
        dv1, dv2 = np.linalg.solve([[a,b],[d,e]], [-c,-f])
        
        return np.array([dth1, dth2, dv1, dv2])
        
def double_pendulum_odeint(pendulum, ts, te, tstep):
    """
    对双摆系统的微分方程组进行数值求解，返回两个小球的X-Y坐标
    """
    t = np.arange(ts, te, tstep)
    track = odeint(pendulum.equations, pendulum.init_status, t)
    th1_array, th2_array = track[:,0], track[:, 1]
    l1, l2 = pendulum.l1, pendulum.l2
    l_x1 = l1*np.sin(th1_array)
    l_y1 = -l1*np.cos(th1_array)
    l_x2 = l_x1 + l2*np.sin(th2_array)
    l_y2 = l_y1 - l2*np.cos(th2_array)
    pendulum.init_status = track[-1,:].copy() #将最后的状态赋给pendulum
    return [l_x1, l_y1, l_x2, l_y2, th1_array, th2_array]

if __name__ == "__main__":    
    pendulum = DoublePendulum(1.0, 1.0, 1.0, 1.0) 
    th1, th2 = 1.0, 0.0
    pendulum.init_status[:2] = th1, th2
    l_x1, l_y1, l_x2, l_y2, th1_array, th2_array = double_pendulum_odeint(pendulum, 0, 20, 0.1)
    fig = plt.close()
    plt.plot(np.linspace(0, 20, 200), th1_array, 'b', label='theta1(t)')
    plt.plot(np.linspace(0, 20, 200), th2_array, 'g', label='theta2(t)')
    plt.legend(loc='best')
    plt.xlabel('t')
    plt.grid()
    plt.show()

In [ ]:
fig2 = plt.figure()
ax = fig2.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
ax.set_aspect('equal')
ax.grid()

line1, = ax.plot([], [], 'o-', lw=2)
line2, = ax.plot([], [], '.-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

def init():
    line1.set_data([], [])
    line2.set_data([], [])
    time_text.set_text('')
    return line1, line2, time_text

def animate(i):
    thisx = [0, x1[i], x2[i]]
    thisy = [0, y1[i], y2[i]]
    l_x = [0, l_x1[i], l_x2[i]]
    l_y = [0, l_y1[i], l_y2[i]]

    line1.set_data(thisx, thisy)
    line2.set_data(l_x, l_y)
    time_text.set_text(time_template % (i*0.1))
    return line1, line2, time_text

ani = animation.FuncAnimation(fig2, animate, np.arange(1, 200),
                              interval=25, blit=True, init_func=init)

ani.save('l_double_pendulum.mp4', fps=10)

In [ ]:
video1 = io.open('l_double_pendulum.mp4', 'r+b').read()
encoded1 = base64.b64encode(video1)
HTML(data='''<video width="500" height="360" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded1.decode('ascii')))
#初始条件相同时，可以看出两种颜色的线条运动轨迹完全相同，证明拉格朗日力学和牛顿力学的结果统一。

#### 作者：赵桪乙